# CR Project : Nanogram solver


- QUENTIN Anthony
- RAMPONT Martin


## Présentation du Projet

Ce projet explore l’application des solveurs SAT et des problèmes de satisfaction de contraintes (CSP) pour résoudre des puzzles logiques, en particulier les **nonogrammes** (également appelés Picross ou Hanjie). L’objectif est de développer un **solveur automatisé** capable de déterminer si un nonogramme donné (2D et 3D) a une solution et, le cas échéant, de trouver cette solution efficacement.

Nous procéderons en deux phases :

1. **Résolution des nonogrammes 2D à l’aide d’un solveur SAT**
2. **Extension de la solution aux nonogrammes 3D**, en tenant compte de l’explosion combinatoire et en justifiant une approche de résolution ligne par ligne.

Les nonogrammes ont gagné en popularité grâce à divers jeux, notamment la série _Nintendo Picross 3D_, sortie pour la première fois en 2009, qui a introduit le concept de nanogrammes en trois dimensions.

---


## Partie 1 : Solveur de Nonogrammes 2D

### Définition du Problème

Un **nonogramme 2D** est constitué d’une grille où chaque ligne et chaque colonne est accompagnée d’une série de nombres. Ces nombres indiquent les blocs de cellules remplies consécutives, séparées par au moins une cellule vide.

#### Exemple d’entrée (lambda.txt) :

```
12 # Nombre de lignes
10 # Nombre de colonnes
2
1 2
1 1
2
1
3
3
2 2 # Exemple pour cette ligne on sait qu'il 2 blocs de 2 cases consécutives séparées par un nombre inconnu de cases vides
2 1
2 2 1
2 3
2 2
2 1
1 3
2 4
3 4
4
3
3
3
2
2
```

#### Exemple de solution (lambda.txt) :

```
.XX.......
X.XX......
X..X......
...XX.....
....X.....
...XXX....
...XXX....
..XX.XX...
..XX..X...
.XX...XX.X
.XX....XXX
XX.....XX.
\...
```

<p align="center">
  <img src="img/lambda.png" alt="lambda solution" width="400"/>
</p>

### Représentation des Données

- **Grille** : Représentée sous forme d’une matrice de valeurs booléennes (`1` pour une case remplie, `0` pour une case vide).
- **Contraintes des lignes et colonnes** : Listes de nombres indiquant les blocs remplis requis.

### Modélisation en SAT (La suite fait référence au code du fichier `picross2d.py` du dossier `picross2d`.)

Chaque cellule `(i, j)` de la grille est représentée comme une variable booléenne. La variable sera positive si la case est noire et négative si la case est blance. Les contraintes sont exprimées sous forme de formules logiques :

#### 1. Analyse des indices et génération des combinaisons possibles

Chaque ligne contient une **séquence de blocs noirs** séparés par au moins un espace blanc. La première étape consiste à analyser les indices de la ligne et à calculer le nombre total de cellules noires (`total_blocks`). Ensuite, on génère **toutes les positions possibles** où ces blocs peuvent être placés en respectant les contraintes de taille et d’espacement.

Pour cela, la fonction `get_intervals(len(blocks), cols - total_blocks)` génère **toutes les combinaisons possibles d'espacement** entre les blocs noirs. Chaque combinaison représente une manière valide de placer ces blocs dans la ligne.

#### 2. Introduction de variables auxiliaires

Pour chaque combinaison possible d'espacement, une **variable auxiliaire** (`cond = aux_var(aux_counter)`) est introduite. Cette variable représente un **choix particulier de placement** pour les blocs noirs dans la ligne. Cela permet de gérer plusieurs configurations possibles de manière efficace dans le solveur SAT.

#### 3. Construction des clauses SAT

Chaque combinaison d'espacement donne lieu à un ensemble de **clauses logiques** qui assurent le bon placement des cases noires et blanches :

- **Contraintes sur les cases blanches** : Avant et après chaque bloc noir, il doit y avoir au moins un espace blanc.
  - Exemple : `[-cond, -var(row, pos, cols)]` signifie que si cette combinaison est choisie (`cond` est vrai), alors la cellule `pos` (`pos` représente l'indice de la colonne lorsqu'on encode les clauses pour les lignes et l'indice de la ligne lorsqu'on encode les clauses pour les colonnes) doit être blanche. La variables `cols` représente le nombre de colonne de la grille.
- **Contraintes sur les cases noires** : Les cases correspondant aux indices doivent être noires.
  - Exemple : `[-cond, var(row, pos, cols)]` garantit que cette cellule doit être noire.

#### 4. Ajout des clauses de disjonction

Enfin, la liste `conditions` contient toutes les variables auxiliaires associées aux placements possibles. La clause `clauses.append(conditions)` est ajoutée pour **garantir qu'au moins une des combinaisons est choisie**.

Le solveur SAT **Gophersat** est utilisé pour résoudre ces contraintes.

#### 5. Exemple Encodage SAT d'une ligne de Nonogram

Considérons une grille **5x5** avec une ligne ayant les indices `"1 2"`. Cela signifie qu'il faut placer **un bloc de 1 case noire** et **un bloc de 2 cases noires**, séparés par au moins une case blanche.

##### Possibilités de placement :

1. `X . X X .`
2. `X . . X X`
3. `. . X . X X`

##### Génération des clauses SAT :

On introduit une **variable auxiliaire** `cond_i` pour chaque placement. Si `cond_1` est vraie, alors la ligne suit la configuration `X . X X .`, et ainsi de suite.  
On ajoute des contraintes pour chaque cellule :

- `(-cond_1 OR var(0,0))` → La première case est noire si `cond_1`
- `(-cond_1 OR -var(0,1))` → La seconde case est blanche si `cond_1`
- `(-cond_1 OR var(0,2))` et `(-cond_1 OR var(0,3))` → Cases 2 et 3 noires si `cond_1`
- `(-cond_1 OR -var(0,4))` → Dernière case blanche si `cond_1`

Enfin, on impose **au moins une configuration valide** : `(cond_1 OR cond_2 OR cond_3)`

### Génération des Combinaisons Possibles pour les Blocs

La fonction `get_intervals` génère toutes les **distributions possibles** des espaces blancs entre ces blocs.

#### Explication du Processus

1. **Définition des Variables**

   - Chaque espace entre les blocs est modélisé comme une variable (`var_0, var_1, ..., var_Nb`).
   - Les premières et dernières valeurs (`var_0` et `var_Nb`) peuvent aller de `0` à `Max` (elles représentent le début et la fin de la ligne).
   - Les valeurs intermédiaires (`var_1` à `var_{Nb-1}`) doivent être au moins `1` pour garantir une séparation correcte entre les blocs.

2. **Ajout des Contraintes**

   - La somme totale des espaces blancs et des blocs doit être égale à la taille maximale de la ligne (`Max`).
   - `ExactSumConstraint(Max)` impose cette contrainte au solveur.

3. **Génération des Solutions**
   - Le solveur trouve **toutes les répartitions possibles** des espaces blancs entre les blocs.
   - Chaque solution est une combinaison valide des positions où les blocs peuvent être placés.

#### Exemple de la ligne avec indices "1 2" et d'une grille 5x5

La fonction prend en entrée `get_intervals(len(blocks)=2, cols - total_blocks = 2)`. Pour rappel `blocks = [1,2] et total_blocks = 1+2 = 3`. Et retourne : `[[0, 2, 0], [1, 1, 0], [0, 1, 1]]`. La première combinaison représente : `X . X X .`

### Affichage de la Grille avec Tkinter

L'interface graphique utilise **Tkinter** pour afficher la grille du Nonogramme. La fonction `draw_nonogram` charge la grille et les indices de lignes et colonnes à partir des fichiers fournis. Elle crée ensuite un **canvas** sur lequel elle dessine les cases de la grille. Les indices des lignes sont affichés à droite de la grille, et ceux des colonnes sont affichés en bas. Chaque case noire est représentée par un rectangle noir, et une fonction d'animation permet de colorier progressivement les cases noires, avec un délai (`time.sleep(0.005)`) entre chaque mise à jour, offrant un effet visuel dynamique.

---


### Deux exemples : lambda et l'hélicopter


In [ ]:
%run picross2d/picross2d.py --input_filename lambda.txt

In [ ]:
%run picross2d/picross2d.py --input_filename helicopter.txt

## Partie 2 : Solveur de Nonogrammes 3D

### Différences avec le 2D

- Au lieu de lignes et colonnes, nous avons maintenant des **plans 3D**.
- **Trois ensembles de contraintes** (lignes, colonnes, profondeur).
- Modification des indices :
  - **Nombre simple** : Un bloc contigu.
  - **Nombre entouré d’un cercle** : Exactement deux blocs distincts.
  - **Nombre entouré d’un carré** : Trois blocs ou plus distincts.


### Types de Clauses en 3D

Les contraintes de résolution des nonogrammes en 3D sont spécifiées par différents types de clauses : plain number, circle, et squared. Ces types correspondent à des configurations spécifiques que nous devons respecter pour chaque pile de blocs dans une grille.


#### 1. Plain Number (Nombre simple)

Un bloc contigu dans une pile, sans séparation.

Exemple : Si un indice indique "3", cela signifie qu'il doit y avoir exactement 3 cases noires contiguës dans une dimension donnée.
Exemples de fichiers à traiter pour cette contrainte :

- LittlePuzzle.txt
- RaspberryPuzzle.txt
- Pyramid.txt


In [ ]:
!python picross3d/picross3d.py LittlePuzzle.txt


In [ ]:
!python picross3d/picross3d.py RaspberryPuzzle.txt
# Il faut un peu d'imagination mais c'est bien censé ressembler à une fraise ;)

In [ ]:
!python picross3d/picross3d.py Pyramid.txt

<p align="center">
  <img src="img/Pyramid.png" alt="lambda solution" width="400"/>
</p>


#### 2. Circle (Entouré d'un cercle)

Les indices entourés de parenthèses indiquent que le nombre de blocs doit être exactement deux dans la pile. Cela diffère des nombres simples car il y a une contrainte stricte de séparation.

Exemples de fichiers à traiter pour cette contrainte :

- CircledPuzzle_small.txt
- CircledPuzzle_big.txt


In [ ]:
!python picross3d/picross3d.py CircledPuzzle_small.txt


In [ ]:
!python picross3d/picross3d.py CircledPuzzle_big.txt

<p align="center">
  <img src="img/CircledPuzzle_big.png" alt="lambda solution" width="400"/>
</p>


#### 3. Squared (Entouré d'un carré)

Un indice entouré de crochets signifie qu'il y a trois blocs ou plus à placer, mais contrairement au "plain number", le nombre exact de blocs peut varier. Cela ajoute une dimension supplémentaire à la résolution, car on ne connaît pas à l'avance le nombre exact de groupes.

Exemple de fichier à traiter pour cette contrainte :

- SquaredPuzzle_small.txt


In [ ]:
!python picross3d/picross3d.py SquaredPuzzle_small.txt

<p align="center">
  <img src="img/SquaredPuzzle_small.png" alt="lambda solution" width="400"/>
</p>


### Étude de l'explosion combinatoire en 3D

Les solveurs SAT modernes sont très performants, mais leur efficacité est limitée par plusieurs facteurs :

- **Nombre de clauses** (taille de la formule CNF)
- **Nombre de variables** (chaque cellule du nonogramme 3D devient une variable binaire)
- **Structure du problème** (certaines formules sont plus difficiles à résoudre que d’autres)
- **Mémoire nécessaire** (les solveurs SAT utilisent des tableaux et structures pour stocker les clauses)

---

#### Capacités des principaux solveurs SAT

Les performances des solveurs SAT dépendent du **nombre de clauses** qu’ils peuvent traiter en **temps raisonnable** (quelques minutes à quelques heures). A titre d'exemple voici un aperçu des performances des principaux solveurs SAT actuels:

| **Solveur**           | **Capacité max (ordre de grandeur)** | **Référence**                        |
| --------------------- | ------------------------------------ | ------------------------------------ |
| **MiniSat**           | ~\(10^5 - 10^6\) clauses             | (historique, peu optimisé)           |
| **Glucose**           | ~\(10^6\) clauses                    | (optimisé pour instances difficiles) |
| **CryptoMiniSat**     | ~\(10^7\) clauses                    | (résout certains SAT industriels)    |
| **CaDiCaL**           | ~\(10^7 - 10^8\) clauses             | (très optimisé)                      |
| **Kissat**            | ~\(10^7 - 10^8\) clauses             | (dernier état de l’art)              |
| **Gophersat**         | ~\(10^6 - 10^7\) clauses             | (utilisé en Python, très rapide)     |
| **PySAT (interface)** | dépend du backend utilisé            | (supporte plusieurs solveurs)        |

---

## Conclusion et Améliorations Possibles

En résolvant les nonogrammes 3D, on peut observer que les configurations plus complexes, en particulier lorsque plusieurs types de contraintes sont mélangés (plain, circle, squared), peuvent entraîner une explosion combinatoire. Les solveurs SAT modernes sont efficaces pour des tailles modérées, mais pour des puzzles plus complexes, l'approche ligne par ligne est nécessaire.

Il existe également des travaux en cours sur des solveurs optimisés pour gérer ces explosions combinatoires, notamment des approches hybrides qui combinent des solveurs SAT avec des heuristiques pour accélérer la recherche de solutions, en particulier dans des puzzles 3D de grande taille. Des projets comme Picross 3D Solver et des recherches sur la génération et la résolution de puzzles 3D contribuent à améliorer les performances de ces solveurs.

### Références

- [Picross 3D Solver](https://github.com/epmjohnston/Picross-3D-Solver)
- Article de recherche : _Generating and solving 3D nonogram puzzles_ (Connor Halford, 2016)

---
